## Ejercicio 1: Función VaR y ES

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

pd.set_option('display.float_format', '{:,.4f}'.format)

In [2]:
def var_stocks(data, n_stocks, conf, long):
    rt = data.pct_change().dropna()
    stock_value = n_stocks * data.iloc[-1]
    portfolio_value = stock_value.sum()
    w = stock_value / portfolio_value
    portfolio_return = np.dot(w, rt.T)

    if long == 1:

        var_pct = np.percentile(portfolio_return, 100-conf)
        cvar_pct = np.abs(portfolio_return[portfolio_return < var_pct].mean())

        var_cash = portfolio_value * np.abs(var_pct)
        cvar_cash = portfolio_value * cvar_pct

    else:
        
        var_pct = np.percentile(portfolio_return, conf)
        cvar_pct = portfolio_return[portfolio_return > var_pct].mean()

        var_cash = portfolio_value * var_pct
        cvar_cash = portfolio_value * cvar_pct

    var_stocks_df = pd.DataFrame({
        "Métrica": ["VaR", "cVaR"],
        "Porcentaje": [np.abs(var_pct), cvar_pct],
        "cash": [var_cash, cvar_cash]
    })

    return var_stocks_df

In [3]:
stocks = ["AAPL", "TSLA", "AMD", "LMT", "JPM"]
data=yf.download(stocks, start="2020-01-01", end="2023-01-01")['Adj Close'][stocks]

n_stocks =[2193, 1211, 3221, 761, 1231]
conf = 95
long = True

[*********************100%***********************]  5 of 5 completed


In [4]:
var_stocks(data, n_stocks, conf, long)

,Métrica,Porcentaje,cash
0,VaR,0.0289,"33,087.2686"
1,cVaR,0.0463,"53,145.1058"


## Ejercicio 2: Rebalanceo de portafolios

In [5]:
rt = data.pct_change().dropna()
stock_value = n_stocks * data.iloc[-1]
portfolio_value = stock_value.sum()
w_original = stock_value / portfolio_value
target = 30000

In [6]:
def get_var(weights):
    portfolio_return = np.dot(weights, rt.T)

    if long == 1:
        var_pct = np.abs(np.percentile(portfolio_return, 100-conf))
        
    else:
        var_pct = np.percentile(portfolio_return, conf)
    
    return portfolio_value * var_pct

def objective(weights):
    return get_var(weights) #np.sum((weights - w_original) ** 2)

# Restricciones
constraints = [
    {"type": "eq", "fun": lambda w: np.sum(w) - 1},
    {"type": "eq", "fun": lambda w: get_var(w) - target}
]

bounds = tuple((0, 1) for _ in range(len(n_stocks)))

# Optimización
result = minimize(
    fun=objective,
    x0=w_original,
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)

opt_weights = result.x

In [7]:
w_df = pd.DataFrame({
    "Peso Original": w_original,
    "Peso Óptimo": opt_weights,
    "Acciones (C/V)" : (opt_weights-w_original) * portfolio_value / data.iloc[-1]
})
w_df.T

Ticker,AAPL,TSLA,AMD,LMT,JPM
Peso Original,0.2459,0.1301,0.1819,0.3063,0.1358
Peso Óptimo,0.1774,0.0706,0.1222,0.4210,0.2088
Acciones (C/V),-611.0887,-553.6475,"-1,057.0180",284.9638,661.5521
